In [598]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "01" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 34
repetitionName = "udrlBC3H"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

90
10.0


In [599]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 123)
(10, 123)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11132\489148831.py:172: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,6,7,8,9,10,11,12,13,14,15,16,17,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,0.6975,0.715,0.6875,0.6325,0.6525,0.6425,0.635,0.6725,0.6625,None,None,None,0.7275,0.725,0.6975,0.655,0.66,0.6575,0.6375,0.6725,0.66,None,None,None,0.7025,0.705,0.6825,0.65,0.675,0.6525,0.6425,0.675,0.665,0.7225,0.6575,0.6625,0.695,0.66,0.6825,0.65,0.6175,0.62,0.675,0.7,0.695,0.7175,0.7175,0.7025,0.69,0.695,0.68,None,None,None,0.7125,0.715,0.6825,0.66,0.6625,0.665,0.64,0.67,0.6625,0.72,0.6375,0.665,0.69,0.6725,0.675,0.64,0.61,0.62,0.68,0.69,0.6875,0.7175,0.7125,0.7025,0.69,0.69,0.68,0.7125,0.6475,0.66,0.6875,0.6525,0.665,0.645,0.6,0.6175,0.675,0.695,0.6875,0.7125,0.7075,0.7025,0.69,0.6975,0.6825,0.705,0.6875,0.6675,0.6925,0.6725,0.6675,0.7,0.675,0.655,0.7025,0.705,0.69
2,0.620833,None,0.558333,0.73125,0.752083,0.747917,0.7375,0.714583,0.702083,0.654167,0.679167,0.677083,0.602083,None,0.552083,0.727083,0.747917,0.747917,0.752083,0.708333,0.7125,0.672917,0.6875,0.679167,0.610417,None,0.55625,0.733333,0.7625,0.727083,0.75,0.727083,0.7125,0.708333,0.666667,0.664583,0.714583,0.691667,0.722917,0.697917,0.68125,0.697917,0.68125,0.620833,0.660417,0.73125,0.747917,0.722917,0.75,0.739583,0.739583,0.733333,0.7375,0.70625,0.610417,None,0.539583,0.729167,0.741667,0.725,0.739583,0.73125,0.7125,0.708333,0.677083,0.664583,0.70625,0.683333,0.729167,0.697917,0.683333,0.702083,0.697917,0.629167,0.660417,0.727083,0.741667,0.727083,0.747917,0.7375,0.7375,0.735417,0.74375,0.710417,0.754167,0.691667,0.710417,0.722917,0.672917,0.71875,0.720833,0.627083,0.666667,0.747917,0.76875,0.733333,0.74375,0.729167,0.710417,0.74375,0.735417,0.73125,0.7125,0.735417,0.714583,0.71875,0.7,0.714583,0.720833,0.689583,0.70625,0.7375,0.739583,0.722917
3,0.55,None,None,0.641667,0.65,0.683333,0.713889,0.744444,0.727778,0.652778,0.705556,0.633333,None,None,None,0.669444,0.666667,0.697222,0.722222,0.755556,0.727778,0.65,0.705556,0.644444,0.555556,None,None,0.652778,0.622222,0.694444,0.744444,0.719444,0.725,0.638889,0.663889,0.616667,0.686111,0.663889,0.683333,0.727778,0.702778,0.702778,0.669444,0.697222,0.627778,0.697222,0.708333,0.719444,0.661111,0.691667,0.688889,0.694444,0.761111,0.7,0.516667,None,None,0.663889,0.627778,0.691667,0.752778,0.725,0.730556,0.647222,0.672222,0.622222,0.672222,0.666667,0.669444,0.722222,0.708333,0.702778,0.663889,0.683333,0.625,0.713889,0.716667,0.713889,0.663889,0.697222,0.686111,0.697222,0.755556,0.688889,0.677778,0.658333,0.688889,0.716667,0.7,0.716667,0.633333,0.65,0.597222,0.702778,0.7,0.713889,0.669444,0.694444,0.666667,0.727778,0.758333,0.702778,0.702778,0.697222,0.688889,0.683333,0.691667,0.652778,0.727778,0.711111,0.683333,0.683333,0.716667,0.702778
4,0.583333,None,0.597917,0.720833,0.71875,0.739583,0.733333,0.735417,0.733333,0.591667,0.595833,0.641667,0.597917,None,0.6125,0.722917,0.708333,0.739583,0.739583,0.73125,0.729167,0.595833,0.591667,0.65,0.597917,None,0.5875,0.7125,0.654167,0.683333,0.73125,0.6625,0.720833,0.660417,0.6125,0.635417,0.708333,0.620833,0.6875,0.725,0.61875,0.697917,0.639583,0.564583,0.625,0.74375,0.729167,0.7375,0.69375,0.691667,0.69375,0.697917,0.697917,0.677083,0.6,None,0.5875,

In [600]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.79 : subject 5, feature:welchDataBC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:welchDataBC-dataFFTCV--dataHR, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:welchDataBC-dataHRCV-BC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.77 : subject 5, feature:welchDataBC-fftDataBC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:welchDataBC-fftDataBC--dataFFTCV, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 5, feature:fftDataBC--dataFFTCV, kernellinear, C:0.5
Accuracy 0.76 : subject 2, feature:fftDataBC--dataFFTCV, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 3, feature:dataHRBC--welchDataBC, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 3, feature:dataHRBC-welchDataBC--dataFFTCV, kernelsigmoid, C:2.5
Accuracy 0.76 : subject 5, feature:fftDataBC-dataFFTCV--dataHR, kernellinear, C:0.5


In [601]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.67634804 0.70435606 0.68449735 0.67418561 0.6908443
 0.64985795 0.67487745 0.67377193 0.65884395]


,6,7,8,9,10,11,12,13,14,15,16,17,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,0.586673,0.566629,0.603955,0.696242,0.700543,0.708144,0.701047,0.705237,0.695953,0.623103,0.647259,0.642517,0.605765,0.564924,0.599038,0.706264,0.699655,0.706783,0.701177,0.705706,0.696125,0.626987,0.646472,0.645629,0.596027,0.546667,0.583659,0.697489,0.684712,0.691458,0.707283,0.686643,0.684066,0.652385,0.641501,0.63665,0.705906,0.67026,0.685547,0.695443,0.666272,0.681197,0.645995,0.622002,0.627493,0.710487,0.71197,0.71022,0.702746,0.700391,0.695802,0.688448,0.70932,0.675463,0.591976,0.555,0.578121,0.70385,0.682304,0.689579,0.710145,0.683864,0.684081,0.653938,0.644771,0.635651,0.70068,0.666275,0.682266,0.696337,0.66678,0.678676,0.649771,0.620088,0.628321,0.714359,0.712231,0.704498,0.700003,0.699641,0.692371,0.689174,0.706947,0.672911,0.703697,0.668872,0.674384,0.700063,0.662496,0.672904,0.656263,0.607378,0.62236,0.711256,0.703068,0.698353,0.699,0.690359,0.681052,0.697127,0.695934,0.673579,0.716256,0.702747,0.688496,0.697816,0.680261,0.674307,0.690022,0.673709,0.666166,0.704916,0.709625,0.698314


Max average accuracy 0.7162556116722784
111
9.0
welchDataBC-fftDataBC--dataFFTCV
linear
C: 0.5


In [602]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-dataHR
1
-dataFFTCV
2
-dataHRCV-BC
3
-fftDataBC
4
-welchDataBC
5
-dataHRBC
6
dataFFTCV--dataHR
7
dataHRCV-BC--dataHR
8
fftDataBC--dataHR
9
welchDataBC--dataHR
10
dataHRBC--dataHR
11
dataHRCV-BC--dataFFTCV
12
fftDataBC--dataFFTCV
13
welchDataBC--dataFFTCV
14
dataHRBC--dataFFTCV
15
fftDataBC--dataHRCV-BC
16
welchDataBC--dataHRCV-BC
17
dataHRBC--dataHRCV-BC
18
welchDataBC--fftDataBC
19
dataHRBC--fftDataBC
20
dataHRBC--welchDataBC
21
dataHRCV-BC-dataFFTCV--dataHR
22
fftDataBC-dataFFTCV--dataHR
23
welchDataBC-dataFFTCV--dataHR
24
dataHRBC-dataFFTCV--dataHR
25
fftDataBC-dataHRCV-BC--dataHR
26
welchDataBC-dataHRCV-BC--dataHR
27
dataHRBC-dataHRCV-BC--dataHR
28
welchDataBC-fftDataBC--dataHR
29
dataHRBC-fftDataBC--dataHR
30
dataHRBC-welchDataBC--dataHR
31
fftDataBC-dataHRCV-BC--dataFFTCV
32
welchDataBC-dataHRCV-BC--dataFFTCV
33
dataHRBC-dataHRCV-BC--dataFFTCV
34
welchDataBC-fftDataBC--dataFFTCV
35
dataHRBC-fftDataBC--dataFFTCV
36
dataHRBC-welchDataBC--dataFFTCV
37
welchDataBC-fftDataBC--dataHRC

In [603]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,6,7,8,9,10,11,12,13,14,15,16,17,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122
0,3.520038,1.699886,3.019773,6.266174,6.304886,6.373295,6.309419,6.347134,6.263573,5.607929,5.825328,5.782652,3.028826,1.694773,2.995189,6.356376,6.296894,6.361048,6.310593,6.351351,6.265126,5.642879,5.818245,5.810657,3.576162,1.093333,2.918295,6.277399,6.162412,6.223119,6.365543,6.179785,6.156591,5.871465,5.77351,5.729848,6.353157,6.032336,6.169924,6.25899,5.996452,6.13077,5.813952,5.598018,5.647437,6.394381,6.407727,6.391982,6.32471,6.303523,6.262222,6.196035,6.383876,6.079167,3.551856,1.11,2.890606,6.334646,6.140732,6.206212,6.391301,6.154773,6.15673,5.885442,5.802942,5.720859,6.306124,5.996477,6.140391,6.267033,6.001023,6.108081,5.847942,5.580795,5.654886,6.42923,6.410076,6.34048,6.300025,6.296768,6.231338,6.202563,6.362525,6.056199,6.33327,6.019848,6.069457,6.300568,5.962462,6.056136,5.906364,5.466402,5.601237,6.401301,6.327614,6.285177,6.290997,6.213232,6.12947,6.274141,6.263409,6.06221,6.446301,6.324722,6.196465,6.280341,6.122348,6.068763,6.210202,6.063384,5.995492,6.344242,6.386629,6.284823


111
9.0
Max total significant accuracy then averaged 0.7162556116722784 
welchDataBC-fftDataBC--dataFFTCV
linear
C: 0.5 


In [604]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
